# PJBL3




Importando as bibliotecas necessárias

In [ ]:
import numpy as np
import cv2, glob
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import datasets, layers, models, losses
from tensorflow.keras.models import Model
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#To ensure reproducibility
#we set the random seed
seed_number = 10
tf.random.set_seed(seed_number)
np.random.seed(seed_number)


Definindo função para plotar o histórico de treinamento das redes

In [ ]:
#Plot a training history
def plot_history(history):
  print(history.history.keys())
  # summarize history for accuracy
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()
  # summarize history for loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()


# Datasets

Fazendo o download do dataset pelo github

In [ ]:
dataset_url = "https://github.com/andrehochuli/teaching/raw/main/ComputerVision/Lecture%2008%20-%20Classification/basesimpsons.zip"
!wget $dataset_url -O simpsons.zip


Criando diretório de treinamento e teste

In [ ]:
!mkdir train
!mkdir test

Descompactando o dataset e movendo os zip para as devidas pastas

In [ ]:
!unzip -q simpsons.zip 
!unzip -q Teste.zip -d test
!unzip -q Treino.zip -d train

Criando uma pasta para cada classe no treinamento e movendo as imagens

In [ ]:
!mkdir train/bart
!mkdir train/family
!mkdir train/homer 
!mkdir train/lisa 
!mkdir train/maggie 
!mkdir train/marge
!mv train/bart* train/bart
!mv train/family* train/family
!mv train/homer* train/homer 
!mv train/lisa* train/lisa 
!mv train/maggie* train/maggie 
!mv train/marge* train/marge 

Criando uma pasta para cada classe no teste e movendo as imagens

In [ ]:
!mkdir test/bart
!mkdir test/family
!mkdir test/homer 
!mkdir test/lisa 
!mkdir test/maggie 
!mkdir test/marge
!mv test/bart* test/bart
!mv test/family* test/family
!mv test/homer* test/homer 
!mv test/lisa* test/lisa 
!mv test/maggie* test/maggie 
!mv test/marge* test/marge 

definindo o diretório de dados e teste

In [ ]:

data_dir = '/content/train'
test_dir = '/content/test'


Definindo um dataset de imagem para o treinamento e o teste

In [ ]:
batch_size_ = 32
input_shape_ = (600,400,3)

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=seed_number,
  image_size=(input_shape_[0], input_shape_[1]),
  batch_size=batch_size_)

val_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  seed=seed_number,  
  image_size=(input_shape_[0], input_shape_[1]),
  batch_size=batch_size_)

Criando a rede de 5 layers e uma layer de output

-Rescaling: 
Uma camada de pré-processamento que redimensiona os valores de entrada para um novo intervalo.

Essa camada redimensiona cada valor de uma entrada (geralmente uma imagem) multiplicando por escala e adicionando offset.

-Conv2D: Camada de convolução 2D (por exemplo, convolução espacial sobre imagens).

Essa camada cria um kernel de convolução que é convoluído com a entrada da camada para produzir um tensor de saídas.

-MaxPooling: Operação de pool máximo para dados espaciais 2D.

Reduz a amostra da entrada ao longo de suas dimensões espaciais (altura e largura) tomando o valor máximo em uma janela de entrada (de tamanho definido por pool_size) para cada canal da entrada. A janela é deslocada a passos largos ao longo de cada dimensão.

In [ ]:
model = models.Sequential()

#32 layers of size 3x3 and Relu Activation
model.add(layers.Rescaling(1./255,input_shape=input_shape_))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
#Max Pooling of Size (2x2)
model.add(layers.MaxPooling2D((2, 2)))


#64 layers of size 3x3 and Relu Activation
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#Max Pooling of Size (2x2)
model.add(layers.MaxPooling2D((2, 2)))

#64 layers of size 3x3 and Relu Activation
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.summary()

Fully Connected:

-Flatten: Achata a entrada. Não afeta o tamanho do batch.

-Dense: Apenas sua camada NN densamente conectada regular.

Dense implementa a operação: output = activation(dot(input, kernel) + bias) onde ativação é a função de ativação por elemento passada como argumento de ativação, kernel é uma matriz de pesos criada pela camada e bias é um vetor de viés criado pela camada (aplicável apenas se use_bias for True).

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(6,activation='softmax'))

model.summary()

#Training

In [ ]:
epochs_ = 20
model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["acc"])
history = model.fit(train_ds, batch_size=batch_size_, epochs=epochs_, validation_data=val_ds)

In [ ]:
plot_history(history)

In [ ]:
pred= model.predict_generator(val_ds)
predicted_class_indices=np.argmax(pred,axis=1)
labels=val_ds.class_names
predictions=[val_ds.class_names[k] for k in predicted_class_indices]
print(predicted_class_indices)
print(labels)
print(predictions)
y = np.concatenate([y for x, y in val_ds], axis=0)
print(confusion_matrix(predicted_class_indices,y))
print(classification_report(y, predicted_class_indices, target_names=labels))

##Data Augmentation



O aumento de dados na análise de dados são técnicas usadas para aumentar a quantidade de dados adicionando cópias ligeiramente modificadas de dados já existentes ou dados sintéticos recém-criados a partir de dados existentes.

In [ ]:
data_augmentation = tf.keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(input_shape_[0],
                                  input_shape_[1],
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
model = tf.keras.Sequential([
  data_augmentation, #Data Augmentation  
  layers.Rescaling(1./255, input_shape=(96, 96, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),  
  layers.Dropout(0.2), #Regularization
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(6, activation='softmax')
])

In [ ]:
epochs_ = 20
model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["acc"])
history = model.fit(train_ds, batch_size=batch_size_, epochs=epochs_, validation_data=val_ds)

In [ ]:
plot_history(history)

In [ ]:
pred= model.predict_generator(val_ds)
predicted_class_indices=np.argmax(pred,axis=1)
labels=val_ds.class_names
predictions=[val_ds.class_names for k in predicted_class_indices]
print(predicted_class_indices)
print(labels)
print(predictions)
y = np.concatenate([y for x, y in val_ds], axis=0)
print(confusion_matrix(predicted_class_indices,y))
print(classification_report(y, predicted_class_indices, target_names=labels))

Transfer Learning

O aprendizado de transferência é um problema de pesquisa em aprendizado de máquina que se concentra em armazenar o conhecimento adquirido ao resolver um problema e aplicá-lo a um problema diferente, mas relacionado. Por exemplo, o conhecimento adquirido ao aprender a reconhecer carros pode ser aplicado ao tentar reconhecer caminhões.

In [ ]:
conv_layers =  tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False,
                                                        input_tensor=tf.keras.layers.Input(input_shape_),
                                                      classes=6)
conv_layers.trainable = False

model = tf.keras.Sequential([  
  conv_layers,    
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(6, activation='softmax')
])

model.summary()

epochs_ = 10
model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["acc"])
history = model.fit(train_ds, batch_size=batch_size_, epochs=epochs_, validation_data=val_ds)

In [ ]:
plot_history(history)

In [ ]:
class_names = val_ds.class_names
num_classes = len(class_names)
print(class_names)
class_names = np.array(class_names)


In [ ]:
pred= model.predict_generator(val_ds)
predicted_class_indices=np.argmax(pred,axis=1)
labels=class_names
predictions=[class_names[k] for k in predicted_class_indices]
print(predicted_class_indices)
print(labels)
print(predictions)
y = np.concatenate([y for x, y in val_ds], axis=0)
print(confusion_matrix(predicted_class_indices,y))
print(classification_report(y, predicted_class_indices, target_names=labels))

Estado da Arte

Adaptado de: https://www.analyticsvidhya.com/blog/2021/06/image-classification-using-convolutional-neural-network-with-python/

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.set_random_seed(2019)

Aqui além das camadas ja usadas, vemos camadas de dropout

Dropout: Aplica Dropout à entrada.

A camada Dropout define aleatoriamente as unidades de entrada para 0 com uma frequência de taxa em cada etapa durante o tempo de treinamento, o que ajuda a evitar o overfitting. As entradas não definidas como 0 são aumentadas em 1/(1 - taxa) de modo que a soma de todas as entradas não seja alterada.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (180,180,3)) ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(550,activation="relu"),      #Adding the Hidden layer
    tf.keras.layers.Dropout(0.1,seed = 2019),
    tf.keras.layers.Dense(400,activation ="relu"),
    tf.keras.layers.Dropout(0.3,seed = 2019),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.4,seed = 2019),
    tf.keras.layers.Dense(200,activation ="relu"),
    tf.keras.layers.Dropout(0.2,seed = 2019),
    tf.keras.layers.Dense(6,activation = "softmax")   #Adding the Output Layer
])

In [ ]:
model.summary()

Agora criamos métricas de recall, precisão e f1-score, a fim de treinar a rede para maximizar essas métricas

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])




Agora usaremos o ImageDataGenerator que gera batches de dados de imagem de tensor com data augmentation em tempo real.



In [ ]:
bs=30         #Setting batch size

from tensorflow.keras.preprocessing.image import ImageDataGenerator 
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
# Flow training images in batches of 20 using train_datagen generator
#Flow_from_directory function lets the classifier directly identify the labels from the name of the directories the image lies in
train_generator=train_datagen.flow_from_directory(data_dir,batch_size=bs,class_mode='categorical',target_size=(180,180), shuffle=False)
# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=bs,
                                                         class_mode  = 'categorical',
                                                         target_size=(180,180) , shuffle=False)

In [ ]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=150 // bs,
                    epochs=40,
                    validation_steps=50 // bs,
                    verbose=2)

In [ ]:
plot_history(history)

Calculando as métricas

In [ ]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(validation_generator, verbose=0)

In [ ]:
loss

In [ ]:
accuracy

In [ ]:
f1_score

In [ ]:
precision

In [ ]:
recall

In [ ]:
validation_generator.reset()

In [ ]:
pred= model.predict_generator(validation_generator)
predicted_class_indices=np.argmax(pred,axis=1)
labels=(validation_generator.class_indices)
labels2=dict((v,k) for k,v in labels.items())
predictions=[labels2[k] for k in predicted_class_indices]
print(predicted_class_indices)
print(labels)
print(predictions)

In [ ]:
len(predicted_class_indices)

In [ ]:
print(confusion_matrix(predicted_class_indices,validation_generator.classes))

In [ ]:
print(classification_report(validation_generator.classes, predicted_class_indices, target_names=labels))

Podemos perceber que a rede usada permite alcançar um resultado ainda maior de f1-score, o que parece alto para um banco de dados de teste desafiador com imagens que podem ser elusivas ao algoritmo.

Ao comparar com a solução de extração de características pode-se dizer que a rede neural costuma performar melhor, visto que pode iterar e descobrir novos padrões no dataset até inperceptíveis para humanos, ao mesmo tempo, isso dificulta em perceber como o algoritmo funciona e como pode-se melhorar ele para identificar exatamente o que queremos.
Na extração de característica, isso é mais óbvio.
